In [4]:
size_max, compress = 500, '100%'

# = = = = = = = = = = = = = = =

from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

from openpyxl import load_workbook
from openpyxl.utils import get_column_letter

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

import requests

from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

import PIL
from io import BytesIO
from openpyxl.drawing.image import Image

# = = = = = = = = = = = = = = =

wb = load_workbook('./crawler.xlsx')
ws = wb.active

for column in range(1, ws.max_column + 1):
    if ws.cell(1, column).value == 'Json_Src':
        column_src = column
    elif ws.cell(1, column).value == 'Picture':
        column_pic = column

print(f'总数量：{ws.max_row - 1}')

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(2, ws.max_row + 1):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global ws

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =
        
        src = json.loads(ws.cell(a, column_src).value).get('0', '')

        # = = = = = = = = = = = = = = =

        if src == '':
            b = 0
            crawler_status = 'ok'
        else:
            try:
                b = 0
                while True:
                    b += 1
                    
                    try:
                        resp = requests.get(src,
                                            headers=get_header(),
                                            proxies=get_proxy(),
                                            timeout=(10, 10))

                        if resp.status_code == 200:
                            break
                    except:
                        continue

                # = = = = = = = = = = = = = = =

                image = PIL.Image.open(BytesIO(resp.content)).convert('RGB')

                # = = = = = = = = = = = = = = =

                if image.width >= image.height and image.width > size_max:
                    image = image.resize((size_max, int(image.height / (image.width / size_max))), PIL.Image.Resampling.LANCZOS)
                elif image.width < image.height and image.height > size_max:
                    image = image.resize((int(image.width / (image.height / size_max)), size_max), PIL.Image.Resampling.LANCZOS)

                # = = = = = = = = = = = = = = =

                buffer = BytesIO()
                if quality == '100%':
                    image.save(buffer, format="PNG")
                else:
                    image.save(buffer, format="JPEG", quality=int(quality.removesuffix('%')))
                buffer.seek(0)
                
                pic = Image(buffer)

                # = = = = = = = = = = = = = = =

                ws.add_image(pic, f'{get_column_letter(column_pic)}{a}')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                pass

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b}] - {src}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(30):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
wb.save(f'''./pic-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''')
print('Done ~')

总数量：306
[状态：ok，尝试次数：1] - https://m.media-amazon.com/images/I/61kyl8dsUvL._AC_SL1500_.jpg
[剩余数量：276] - [当前时间：14:49:08]

[状态：ok，尝试次数：1] - https://m.media-amazon.com/images/I/61IkbPc60-L._AC_SL1500_.jpg
[剩余数量：275] - [当前时间：14:49:09]

[状态：ok，尝试次数：1] - https://m.media-amazon.com/images/I/61WUYjts27L._AC_SL1500_.jpg
[剩余数量：274] - [当前时间：14:49:09]

[状态：ok，尝试次数：1] - https://m.media-amazon.com/images/I/61-MIUeY2-L._AC_SL1500_.jpg
[剩余数量：273] - [当前时间：14:49:09]

[状态：ok，尝试次数：1] - https://m.media-amazon.com/images/I/619Iu16eIWL._AC_SL1500_.jpg
[剩余数量：272] - [当前时间：14:49:10]

[状态：ok，尝试次数：1] - https://m.media-amazon.com/images/I/61dgQV2P2CL._AC_SL1500_.jpg
[剩余数量：271] - [当前时间：14:49:10]

[状态：ok，尝试次数：1] - https://m.media-amazon.com/images/I/61cRLYrJJiL._AC_SL1396_.jpg
[剩余数量：270] - [当前时间：14:49:10]

[状态：ok，尝试次数：1] - https://m.media-amazon.com/images/I/61iAqAnid+L._AC_SL1500_.jpg
[剩余数量：269] - [当前时间：14:49:10]

[状态：ok，尝试次数：1] - https://m.media-amazon.com/images/I/61Coc3uJvnL._AC_SL1459_.jpg
[剩余数量：268] - [当前时间：14:4